Import necessary libraries

In [1]:
import time
import matplotlib.pyplot as plt
import scipy
import numpy as np
from PIL import Image
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os

# Notebook code based on learn.co Convolutional Neural Networks Code Along

# Pre-Processing

Assign folder paths to variables

In [2]:
train_folder = 'data/seg_train/seg_train'
test_folder = 'data/seg_test/seg_test'
validation_folder = 'data/seg_pred/seg_pred'
im_classes = ['buildings','forest', 'glacier', 'mountain', 'sea', 'street']

Create lists of image names for the training, testing and validation sets and find the length of each set

In [3]:
train_imgs = []
test_imgs = []
validation_imgs = []

for im_class in im_classes:
    train_imgs.extend([file for file in os.listdir(train_folder+'/'+im_class) if file.endswith('.jpg')])
    test_imgs.extend([file for file in os.listdir(test_folder+'/'+im_class) if file.endswith('.jpg')])
    
validation_imgs = [file for file in os.listdir(validation_folder) if file.endswith('.jpg')]
    
train_batch_size = len(train_imgs)
test_batch_size = len(test_imgs)
validation_batch_size = len(validation_imgs)

Create variables with image data

In [5]:
test_generator = ImageDataGenerator().flow_from_directory(
        test_folder, 
        target_size=(64, 64), batch_size = test_batch_size) 

train_generator = ImageDataGenerator().flow_from_directory(
        train_folder, 
        target_size=(64, 64), batch_size = train_batch_size)

Found 3000 images belonging to 6 classes.
Found 14034 images belonging to 6 classes.


Create the data sets

In [6]:
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)


Explore datasets

In [7]:
m_train = train_images.shape[0]
num_px = train_images.shape[1]
m_test = test_images.shape[0]

print ("Number of training samples: " + str(m_train))
print ("Number of testing samples: " + str(m_test))
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))
print ("test_images shape: " + str(test_images.shape))
print ("test_labels shape: " + str(test_labels.shape))


Number of training samples: 14034
Number of testing samples: 3000
train_images shape: (14034, 64, 64, 3)
train_labels shape: (14034, 6)
test_images shape: (3000, 64, 64, 3)
test_labels shape: (3000, 6)


Reshape image data

In [17]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)

print(train_img.shape)
print(test_img.shape)

(14034, 12288)
(3000, 12288)


Reshape image labels

In [18]:
train_y = np.reshape(train_labels[:,0], (train_batch_size,1))
test_y = np.reshape(test_labels[:,0], (test_batch_size,1))


# Baseline Neural Network

Build neural network

In [46]:
from keras import models
from keras import layers
model = models.Sequential()
model.add(layers.Dense(10, activation='relu', input_shape=(12288,))) # 2 hidden layers
model.add(layers.Dense(10, activation='relu', input_shape=(10,)))
model.add(layers.Dense(10, activation='relu', input_shape=(10,)))
model.add(layers.Dense(1, activation='sigmoid', input_shape=(10,)))

Run neural network

In [47]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

histoire = model.fit(train_img,
                    train_y,
                    epochs=5,
                    batch_size=32,
                    validation_data=(test_img, test_y)
                                    )

Epoch 1/5
439/439 [==============================] - 6s 13ms/step - loss: nan - accuracy: 0.8391 - val_loss: nan - val_accuracy: 0.8543
Epoch 2/5
439/439 [==============================] - 5s 12ms/step - loss: nan - accuracy: 0.8439 - val_loss: nan - val_accuracy: 0.8543
Epoch 3/5
439/439 [==============================] - 5s 11ms/step - loss: nan - accuracy: 0.8439 - val_loss: nan - val_accuracy: 0.8543
Epoch 4/5
439/439 [==============================] - 5s 11ms/step - loss: nan - accuracy: 0.8439 - val_loss: nan - val_accuracy: 0.8543
Epoch 5/5
439/439 [==============================] - 5s 12ms/step - loss: nan - accuracy: 0.8439 - val_loss: nan - val_accuracy: 0.8543


# Convolutional Neural Network

Build CNN

In [48]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(64 ,64,  3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['acc'])

Run CNN

In [49]:
history = model.fit(train_images,
                    train_y,
                    epochs=2,
                    batch_size=32,
                    validation_data=(test_images, test_y))

Epoch 1/2
439/439 [==============================] - 137s 313ms/step - loss: 4456008.0000 - acc: 0.8347 - val_loss: 0.4666 - val_acc: 0.8543
Epoch 2/2
439/439 [==============================] - 144s 327ms/step - loss: 0.4558 - acc: 0.8439 - val_loss: 0.4297 - val_acc: 0.8543
